#**MP1_report_59**

##**組員與分工**

組員: 林群佑, 刁一平

|      工作項目      |      分工     |
|:------------------:|:-------------:|
| Trace Code (a),(b) |     林群佑    |
|   Trace Code (c),(d)   |     刁一平    |
|      功能實作與除錯      |     林群佑    |

##1. Trace code

###(a)SC_Halt

首先使用 ../build.linux/nachos -e halt -d c 指令追蹤halt具體調用哪些程式部分

In [ ]:
[os24team59@localhost test]$ ../build.linux/nachos -e halt -d c
halt
In Thread::Sleep, Sleeping thread: main, 20
In Semaphore::P(), 30
In Thread::Sleep, Sleeping thread: postal worker, 30
//由於中間 Tick 40-Tick 52 並沒有實際執行其他函式，故省略
In Machine::Run(), return from OneTick == Tick 53 ==
In Machine::Run(), into OneInstruction == Tick 53 ==
In Machine::OneInstruction, RaiseException(SyscallException, 0), 53
In ExceptionHandler(), Received Exception 1 type: 0, 53

####**Machine::Run()**

當系統執行 system call 時便會調用 Machine::Run() 來處理



In [ ]:
void Machine::Run() {
    Instruction *instr = new Instruction;  // storage for decoded instruction
    if (debug->IsEnabled('m')) {
        cout << "Starting program in thread: " << kernel->currentThread->getName();
        cout << ", at time: " << kernel->stats->totalTicks << "\n";
    }
    kernel->interrupt->setStatus(UserMode);
    for (;;) {
        DEBUG(dbgTraCode, "In Machine::Run(), into OneInstruction "
                              << "== Tick " << kernel->stats->totalTicks << " ==");
        OneInstruction(instr);
        DEBUG(dbgTraCode, "In Machine::Run(), return from OneInstruction  "
                              << "== Tick " << kernel->stats->totalTicks << " ==");

        DEBUG(dbgTraCode, "In Machine::Run(), into OneTick "
                              << "== Tick " << kernel->stats->totalTicks << " ==");
        kernel->interrupt->OneTick();
        DEBUG(dbgTraCode, "In Machine::Run(), return from OneTick "
                              << "== Tick " << kernel->stats->totalTicks << " ==");
        if (singleStep && (runUntilTime <= kernel->stats->totalTicks))
            Debugger();
    }
}

這部分檢查有哪些指令輸入，然後將指令丟到OneInstruction()中執行

####**Machine::OneInstruction()**

這部分模擬CPU處理各種 system call 指令

In [ ]:
case OP_SYSCALL:
  DEBUG(dbgTraCode, "In Machine::OneInstruction, RaiseException(SyscallException, 0), " << kernel->stats->totalTicks);
  RaiseException(SyscallException, 0);
  return;
//由於其他部分在過程中使用不到故省略

透過 In Machine::OneInstruction, RaiseException(SyscallException, 0), 53 我們可以瞭解到執行 halt 使用到了 OP_SYSCALL 的部分，會呼叫 RaiseException(SyscallException, 0); 去處理異常的情況

####**RaiseException()**

首先呼叫 kernal->interrupt->setStatus(SystemMode) 代表了此時從 User Mode 轉變為 Kernal Mode

接著呼叫 ExceptionHandler() 去處理例外狀況，處理完後呼叫

kernal->interrupt->setStatus(UserMode) 代表了 ExceptionHander() 執行完後，要從 Kernal Mode 轉回 User Mode


In [ ]:
void Machine::RaiseException(ExceptionType which, int badVAddr) {
    DEBUG(dbgMach, "Exception: " << exceptionNames[which]);
    registers[BadVAddrReg] = badVAddr;
    DelayedLoad(0, 0);  // finish anything in progress
    kernel->interrupt->setStatus(SystemMode);
    ExceptionHandler(which);  // interrupts are enabled at this point
    kernel->interrupt->setStatus(UserMode);
}

####**ExceptionHandler()**

In [ ]:
void ExceptionHandler(ExceptionType which) {
    char ch;
    int val;
    int type = kernel->machine->ReadRegister(2);
    int status, exit, threadID, programID, fileID, numChar;
    DEBUG(dbgSys, "Received Exception " << which << " type: " << type << "\n");
    DEBUG(dbgTraCode, "In ExceptionHandler(), Received Exception " << which << " type: " << type << ", " << kernel->stats->totalTicks);
    switch (which) {
        case SyscallException:
            switch (type) {
                case SC_Halt:
                    DEBUG(dbgSys, "Shutdown, initiated by user program.\n");
                    SysHalt();
                    cout << "in exception\n";
                    ASSERTNOTREACHED(); //屬於是debug中的功能，若上面程式失效會輸出在哪行失效，在debug.h中
                    //由於其他case並不會執行故省略
    }
    ASSERTNOTREACHED();
}

由 In ExceptionHandler(), Received Exception 1 type: 0, 53

可以得知 which=1, type=0 而各自定義分別是 SyscallException 與 SC_Halt

而具體處理 SyscallException 的程式在 SysHalt() 中

In [ ]:
enum ExceptionType {
    NoException, // Everything ok!
    SyscallException, // A program executed a system call.
    PageFaultException, // No valid translation found
    ReadOnlyException, // Write attempted to page marked
    BusErrorException, // Translation resulted in an
    AddressErrorException, // Unaligned reference or one that
    OverflowException, // Integer overflow in add or sub.
    IllegalInstrException, // Unimplemented or reserved instr.
    NumExceptionTypes
};

在machine.h的標頭檔中，我們可以找到 which=1 的定義是 SyscallException

In [ ]:
#define SC_Halt 0
#define SC_Exit 1
#define SC_Exec 2
#define SC_Join 3
#define SC_Create 4
#define SC_Remove 5
// #define SC_Open	6
// #define SC_Read	7
// #define SC_Write	8
#define SC_Seek 9
// #define SC_Close	10
#define SC_ThreadFork 11
#define SC_ThreadYield 12
#define SC_ExecV 13
#define SC_ThreadExit 14
#define SC_ThreadJoin 15
#define SC_PrintInt 16
#define SC_Add 42
#define SC_MSG 100

在 syscall.h 的標頭黨中，我們可以發現 type=0 的定義是 SC_Halt

####**SysHalt()**

可以看到具體實現方式是呼叫位於 kernel 模式下的 interrupt 的 Halt() 函式

In [ ]:
void SysHalt() {
    kernel->interrupt->Halt();
}

####**Interrupt::Halt()**

我們可以很明顯的看到，具體執行的就是把 kernel 刪除，由於 kernel 包含了所有程式與函數，故刪除後自然就停止了

In [ ]:
void Interrupt::Halt() {
#ifndef NO_HALT_STAT
    cout << "Machine halting!\n\n";
    cout << "This is halt\n";
    kernel->stats->Print();
#endif
    delete kernel;  // Never returns.
}

###(b) SC_Create

首先使用 ../build.linux/nachos -e createFile -d c 指令追蹤 createFile 具體調用哪些程式

In [ ]:
[os24team59@localhost test]$ ../build.linux/nachos -e createFile -d c
createFile
In Thread::Sleep, Sleeping thread: main, 20
In Semaphore::P(), 30
In Thread::Sleep, Sleeping thread: postal worker, 30
//由於中間 Tick 40-Tick 54 並沒有實際執行其他函式，故省略
In Machine::Run(), return from OneTick == Tick 55 ==
In Machine::Run(), into OneInstruction == Tick 55 ==
In Machine::OneInstruction, RaiseException(SyscallException, 0), 55
In ExceptionHandler(), Received Exception 1 type: 4, 55
In Machine::Run(), return from OneInstruction  == Tick 55 ==
In Machine::Run(), into OneTick == Tick 55 ==
//由於中間 Tick 56-Tick 67 並沒有實際執行其他函式，故省略
In Machine::Run(), return from OneTick == Tick 68 ==
In Machine::Run(), into OneInstruction == Tick 68 ==
In Machine::OneInstruction, RaiseException(SyscallException, 0), 68
In ExceptionHandler(), Received Exception 1 type: 100, 68
Success on creating file0.test

####**ExceptionHandler()**

In [ ]:
void ExceptionHandler(ExceptionType which) {
    char ch;
    int val;
    int type = kernel->machine->ReadRegister(2);
    int status, exit, threadID, programID, fileID, numChar;
    DEBUG(dbgSys, "Received Exception " << which << " type: " << type << "\n");
    DEBUG(dbgTraCode, "In ExceptionHandler(), Received Exception " << which << " type: " << type << ", " << kernel->stats->totalTicks);
    switch (which) {
        case SyscallException:
            switch (type) {
                case SC_Create:
                    val = kernel->machine->ReadRegister(4);
                    {
                        char *filename = &(kernel->machine->mainMemory[val]);
                        // cout << filename << endl;
                        status = SysCreate(filename);
                        kernel->machine->WriteRegister(2, (int)status);
                    }
                    kernel->machine->WriteRegister(PrevPCReg, kernel->machine->ReadRegister(PCReg));
                    kernel->machine->WriteRegister(PCReg, kernel->machine->ReadRegister(PCReg) + 4);
                    kernel->machine->WriteRegister(NextPCReg, kernel->machine->ReadRegister(PCReg) + 4);
                    return;
                    ASSERTNOTREACHED(); //屬於是debug中的功能，若上面程式失效會輸出在程式的哪裡失效，在debug.h中
                //由於其他case沒用到故省略
    }
    ASSERTNOTREACHED();
}

由 In ExceptionHandler(), Received Exception 1 type: 4, 55 得知

which=1, type=4 分別是 SyscallException 與 SC_Create (參考上題 ExceptionHandler()的補充說明可得知)

這個函式執行了從暫存器4中讀取相應指令位置，然後到主記憶體中取出實際資料路徑，然後呼叫 SysCreate() 來完成實際的處理
做完 SysCreate() 後，將返回的值存到記憶體之中

####**SysCreate()**

In [ ]:
int SysCreate(char *filename) {
    // return value
    // 1: success
    // 0: failed
    return kernel->fileSystem->Create(filename);
}

執行位於 fileSystem.cc 的 Create() 函式

####**FileSystem::Create()**

In [ ]:
bool Create(char *name) {
  int fileDescriptor = OpenForWrite(name);
  if (fileDescriptor == -1)
    return FALSE;
  Close(fileDescriptor);
  return TRUE;
}

此函式其實就是呼叫一次 OpenForWrite 函式來去模擬創建檔案的行為

###(c) SC_Print_Int

ExceptionHandler前面的地方已由(A)HC_Halt提及，所以從ExceptionHandler開始tracecode

####**ExceptionHandler()**

 根據不同exception類別, 採取不同行動

In [ ]:
void ExceptionHandler(ExceptionType which) {
    char ch;
    int val;
    int type = kernel->machine->ReadRegister(2); # 將 r2 保存的 system call type 存進 type 變數中
    int status, exit, threadID, programID, fileID, numChar;
    switch (which) { # 辨別system call 或 其他expection
        case SyscallException:
            switch (type) {  # 辨別system call的type，並執行該type處理之事
                case SC_PrintInt:
                    DEBUG(dbgSys, "Print Int\n");
                    val = kernel->machine->ReadRegister(4);
                    DEBUG(dbgTraCode, "In ExceptionHandler(), into SysPrintInt, " << kernel->stats->totalTicks);
                    SysPrintInt(val);
                    DEBUG(dbgTraCode, "In ExceptionHandler(), return from SysPrintInt, " << kernel->stats->totalTicks);
                    // Set Program Counter
                    kernel->machine->WriteRegister(PrevPCReg, kernel->machine->ReadRegister(PCReg));
                    kernel->machine->WriteRegister(PCReg, kernel->machine->ReadRegister(PCReg) + 4);
                    kernel->machine->WriteRegister(NextPCReg, kernel->machine->ReadRegister(PCReg) + 4);
                    return;
                    ASSERTNOTREACHED(); # debug用
                    break;
                }
                     # 其餘case省略

從r4中讀取system call之parameter  (val) 然後以此呼叫 SysPrintInt , 在做完SysPrintInt之後，更新program counter









 void Machine::WriteRegister(int num, int value): 在 machine.cc 的 function, 其功能為將 value 寫進 register[num] 中

####**SysPrintInt()** 位於 ksycall.h中

In [ ]:
//userprog/ksyscall.h
void SysPrintInt(int val) {
    DEBUG(dbgTraCode, "In ksyscall.h:SysPrintInt, into synchConsoleOut->PutInt, " << kernel->stats->totalTicks);
    kernel->synchConsoleOut->PutInt(val);
    DEBUG(dbgTraCode, "In ksyscall.h:SysPrintInt, return from synchConsoleOut->PutInt, " << kernel->stats->totalTicks);
}

傳入val, 呼叫synchConsoleOut.cc的PutInt()函數

####**SynchConsoleOutput::PutInt()**

In [ ]:
void SynchConsoleOutput::PutInt(int value) {
    char str[15];
    int idx = 0;
    // sprintf(str, "%d\n\0", value);  the true one
    sprintf(str, "%d\n\0", value);  // simply for trace code, # 將 value 轉成字串str
    lock->Acquire(); #利用lock->Acquire()鎖定, 並且只有現在這個thread可以取得該鎖，確保輸出過程不被截斷
    do { # 將 str 的每一個字元逐一傳送
        DEBUG(dbgTraCode, "In SynchConsoleOutput::PutChar, into consoleOutput->PutChar, " << kernel->stats->totalTicks);
        consoleOutput->PutChar(str[idx]);
        DEBUG(dbgTraCode, "In SynchConsoleOutput::PutChar, return from consoleOutput->PutChar, " << kernel->stats->totalTicks);
        idx++;

        DEBUG(dbgTraCode, "In SynchConsoleOutput::PutChar, into waitFor->P(), " << kernel->stats->totalTicks);
        waitFor->P(); # 每次輸出一個字符後，會產生一個等待操作來確保輸出完成,這裡使用了 waitFor->P() 來讓thread等待輸出完成的信號
        DEBUG(dbgTraCode, "In SynchConsoleOutput::PutChar, return form  waitFor->P(), " << kernel->stats->totalTicks);
    } while (str[idx] != '\0');
    lock->Release(); # 當所有字元都被成功輸出後，通過 lock->Release() 解除(釋放)鎖定
}

除錯信息

In [ ]:
In SynchConsoleOutput::PutChar, into consoleOutput->PutChar, 73
In SynchConsoleOutput::PutChar, return from consoleOutput->PutChar, 73
In SynchConsoleOutput::PutChar, into waitFor->P(), 73
In Semaphore::P(), 73
In Thread::Sleep, Sleeping thread: add, 73
In Interrupt::Idle, into CheckIfDue, 73
In Interrupt::CheckIfDue, into callOnInterrupt->CallBack, 100

為了確保多個執行緒不會同時寫入，這裡使用了鎖機制 lock->Acquire() 來保護對輸出的權限, 當一個執行緒獲得了這個鎖後，其他執行緒需要等待該鎖release後才能繼續

P(): 能夠讓 thread sleep, 禁⽌進⾏其他 thread, 直到接到 call back

由下面output P()與V()的作用可以等待前一個char輸出完成後，再去執行輸出下一個char

####**SynchConsoleOutput::PutChar()**

In [ ]:
SynchConsoleOutput::PutChar(char ch)
{
    lock->Acquire();
    consoleOutput->PutChar(ch);
    waitFor->P();
    lock->Release();
}

與SynchConsoleOutput::PutInt()差別在於傳入參數為字元

####**ConsoleOutput::PutChar**

In [ ]:
ConsoleOutput::PutChar(char ch)
{
  ASSERT(putBusy == FALSE); #若putBusy == TRUE，則終止
  WriteFile(writeFileNo, &ch, sizeof(char)); #將字元 ch 寫入simulated display
  putBusy = TRUE; #一旦字元被寫入，將 putBusy 設置為 TRUE
  kernel->interrupt->Schedule(this, ConsoleTime, ConsoleWriteInt); #進入interrupt.cc裡的Schedule()
}


這個函數的目的是將字元輸出到simulated display，並安排interrupt來完成輸出操作

使用 ASSERT 確保沒有正在進行的輸出，通過sysdep.cc裡的 WriteFile 使用unix的
write()，來將字元寫入檔案中，設置 putBusy 標記輸出狀態，

接著呼叫Schedule() 來安排 interrupt來完成整個輸出過程

ConsoleTime 為 I/O 機器讀寫一個
字元所需的時間，讓CPU排程interrupt，在將來I/O結束後再回去執行原來得位置

這種設計能夠有效地避免輸出過程的阻塞(busy waiting)，允許系統在輸出操作進行時同時處理其他任務，並保證輸出過程的同步和正確性。








####**Interrupt::Schedule()**

In [ ]:
Interrupt::Schedule(CallBackObj *toCall, int fromNow, IntType type)
{
    int when = kernel->stats->totalTicks + fromNow; #現在時間 + 多久後
    PendingInterrupt *toOccur = new PendingInterrupt(toCall, when, type);

    DEBUG(dbgInt, "Scheduling interrupt handler the " << intTypeNames[type] << " at time = " << when);
    ASSERT(fromNow > 0); #確保fromNow⼤於0，若等於0就當下立刻就要中止，因為並不知道當前是否有interrupt正在執⾏，避免衝突

    # pending 是一個sorted list，用來處存interrupt種類
    pending->Insert(toOccur); #使用 Insert 將新的interrupt事件插入list中
}



toCall: interrupt時要被執行的對象  

fromNow: interrupt多久後要發⽣

type: interrupt是由哪個設備產生的

when: interrupt發生的時間點  

Interrupt::Schedule是用來安排一個未來的 interrupt ,  
根據totalTicks + fromNow， 計算出 interrupt 的觸發時間，  
接著將 interrupt 事件(toOccur)插入到 interrupt pending list中,依時間來排程  

這樣在達到指定時間時，系統會觸發該 interrupt

####**Machine::Run()**

In [ ]:
void Machine::Run() {
    Instruction *instr = new Instruction;  // storage for decoded instruction
    if (debug->IsEnabled('m')) {
        cout << "Starting program in thread: " << kernel->currentThread->getName();
        cout << ", at time: " << kernel->stats->totalTicks << "\n";
    }
    kernel->interrupt->setStatus(UserMode); # 將系統設為user mode
        DEBUG(dbgTraCode, "In Machine::Run(), into OneInstruction "
                              << "== Tick " << kernel->stats->totalTicks << " ==");
        OneInstruction(instr);
        DEBUG(dbgTraCode, "In Machine::Run(), return from OneInstruction  "
                              << "== Tick " << kernel->stats->totalTicks << " ==");

        DEBUG(dbgTraCode, "In Machine::Run(), into OneTick "
                              << "== Tick " << kernel->stats->totalTicks << " ==");
        kernel->interrupt->OneTick();
        DEBUG(dbgTraCode, "In Machine::Run(), return from OneTick "
                              << "== Tick " << kernel->stats->totalTicks << " ==");
        if (singleStep && (runUntilTime <= kernel->stats->totalTicks))
            Debugger();
    }
}

前面已有提及

檢查有哪些指令輸入，然後將指令丟到OneInstruction()中執行,

通過 OneTick() 來推進時間⼀個單位, 並檢查有沒有 pending interrupts 待調⽤, 有則進⾏interrupt處理

####**Interrupt::OneTick()**

In [ ]:
Interrupt::OneTick()
{
    MachineStatus oldStatus = status;
    Statistics *stats = kernel->stats;

// advance simulated time
    if (status == SystemMode) {
      stats->totalTicks += SystemTick; # SystemTick = 10
      stats->systemTicks += SystemTick;
    } else {
      stats->totalTicks += UserTick; # UserTick = 1
      stats->userTicks += UserTick;
    }
    DEBUG(dbgInt, "== Tick " << stats->totalTicks << " ==");

// check any pending interrupts are now ready to fire  # ChangeLevel: 改變interupt狀態
    ChangeLevel(IntOn, IntOff); // first, turn off interrupts
                // (interrupt handlers run with
                // interrupts disabled)
    CheckIfDue(FALSE);      // check for pending interrupts
    ChangeLevel(IntOff, IntOn); // re-enable interrupts
    if (yieldOnReturn) {    // if the timer device handler asked
                    // for a context switch, ok to do it now
    yieldOnReturn = FALSE;
    status = SystemMode;        // yield is a kernel routine
    kernel->currentThread->Yield();
    status = oldStatus;
    }
}

用來模擬系統中時間的推進，並檢查是否有需要觸發的 pending interrupts  
根據當前系統處於system mode或user mode，來決定kernel的推進時間,SystemTick（10）或UserTick（1）  

進入CheckIfDue去檢查是否⽬前有需要執⾏的的pending interrupt，  

若有,它會短暫禁用interrupt，檢查完後再透過ChangeLevel(IntOn, IntOff)重新啟用interupt,  
避免處理interrupt時同時有其他interrupts產生

如果當前要求進行context switch，則將 yieldOnReturn設為FALSE,並呼叫currentThread的 Yield(),當此thread⼜可以執⾏時，則恢復 status
   
Yield(): 將 thread 放到 ready list 的最後, 將 cpu 分給下一個thread

####**Interrupt::CheckIfDue()**

In [ ]:
bool Interrupt::CheckIfDue(bool advanceClock) {
    PendingInterrupt *next;
    Statistics *stats = kernel->stats;

    ASSERT(level == IntOff);  // interrupts need to be disabled, # 確保interrupt關掉
                              // to invoke an interrupt handler


    #若有設定interrupt debug flag，則呼叫DumpState()印出⽬前的pending interrupts
    if (debug->IsEnabled(dbgInt)) {
        DumpState();
    }
    if (pending->IsEmpty()) {  // no pending interrupts # 判斷是否有待辦 interrupt
        return FALSE;
    }
    next = pending->Front(); # 取出最前⾯的待辦interrupt

    if (next->when > stats->totalTicks) { #若要發生interrupt的時間還未到,使⽤advanceClock判斷是否將totakTicks往後移到interrupt的發⽣時間
        if (!advanceClock) {  // not time yet
            return FALSE;
        } else {  // advance the clock to next interrupt
            stats->idleTicks += (next->when - stats->totalTicks);
            stats->totalTicks = next->when;
            // UDelay(1000L); // rcgood - to stop nachos from spinning.
        }
    }

    DEBUG(dbgInt, "Invoking interrupt handler for the ");
    DEBUG(dbgInt, intTypeNames[next->type] << " at time " << next->when);

    if (kernel->machine != NULL) { #若kernel 未被刪除,則使用DelayedLoad(0, 0) 結束當前運⾏的東西
        kernel->machine->DelayedLoad(0, 0);
    }

    inHandler = TRUE; # 處理interrupt

    do {  #不停地執行interrupt並刪除，直到所有目前時間點或以前的interrupt都被執行完畢
        next = pending->RemoveFront();  // pull interrupt off list
        DEBUG(dbgTraCode, "In Interrupt::CheckIfDue, into callOnInterrupt->CallBack, " << stats->totalTicks);
        next->callOnInterrupt->CallBack();  // call the interrupt handler
        DEBUG(dbgTraCode, "In Interrupt::CheckIfDue, return from callOnInterrupt->CallBack, " << stats->totalTicks);
        delete next;
    } while (!pending->IsEmpty() && (pending->Front()->when <= stats->totalTicks)); #直到pending為空或是下一個interrupt大於目前時間
    inHandler = FALSE; #處理完interrupt
    return TRUE;
}


檢查是否有已排程的interupt需要觸發，並在條件滿足時執行interupt處理程序


如果沒有interupt需要處理，則可能選擇直接推進系統時間到下一個interupt的觸發點

####**ConsoleOutput::CallBack()**

In [ ]:
void ConsoleOutput::CallBack() {
    DEBUG(dbgTraCode, "In ConsoleOutput::CallBack(), " << kernel->stats->totalTicks);
    putBusy = FALSE; #將putBusy設回原本
    kernel->stats->numConsoleCharsWritten++; #紀錄已經寫入顯示器的字元數
    callWhenDone->CallBack();
}


寫入完後將putBusy設為FALSE,


代表能夠輸出下一個 character, 並再次呼叫 CallBack()

在成功輸出後更新已顯示character數目

####**SynchConsoleOutput::CallBack()**

In [ ]:
void SynchConsoleOutput::CallBack() {
    DEBUG(dbgTraCode, "In SynchConsoleOutput::CallBack(), " << kernel->stats->totalTicks);
    waitFor->V();
}

waitFor->V(): 釋放一個信號, 會喚醒在SynchConsoleOutput::PutInt P() 等待的 thread,讓其他的thread可以執行

 i.e: SynchConsoleOutput::PutInt P()執行下一行


###(d) Makefile

In [ ]:
nclude Makefile.dep

CC = $(GCCDIR)gcc
AS = $(GCCDIR)as
LD = $(GCCDIR)ld

INCDIR =-I../userprog -I../lib
CFLAGS = -g -G 0 -c $(INCDIR) -B/usr/bin/local/nachos/lib/gcc-lib/decstation-ultrix/2.95.2/ -B/usr/bin/local/nachos/decstation-ultrix/bin/


 CC: Compiler    

 AS: Assembler    

 LD: Linker

 INCDIR: Header檔的⽬錄

In [ ]:
ifeq ($(hosttype),unknown) # 檢查 $(hosttype) 是否等於 "unknown"
PROGRAMS = unknownhost
else
# change this if you create a new test program!
PROGRAMS = add halt createFile LotOfAdd # 設置program 變數
endif

all: $(PROGRAMS)

start.o: start.S ../userprog/syscall.h  # start.o⽬標依賴於start.S及syscall.h
	$(CC) $(CFLAGS) $(ASFLAGS) -c start.S # -c: ⽣成.o檔,⽽不是經由linker連結成可執⾏檔

halt.o: halt.c # 表示 halt.o 依賴於 halt.c,所以每當 halt.c 被修改或更新時，halt.o 需要重新make
	$(CC) $(CFLAGS) -c halt.c
halt: halt.o start.o
	$(LD) $(LDFLAGS) start.o halt.o -o halt.coff
	$(COFF2NOFF) halt.coff halt

all: $(PROGRAMS)：是一個變數，包含所有要編譯的程序名稱,確保在執行 make 時，會編譯所有列出的程序。

CFLAGS: 這是一個變數，用來指定編譯器的flags

start.o ： start.S

使用compler來編譯件 start.S，並包含 syscall.h 標頭文件，輸出為 start.o,

halt.o ： halt.c  
使用complier來編譯 halt.c，輸出為 halt.o,

halt:  halt.o start.o  

將 start.o 和 halt.o 連結起來，生成 halt.coff 可執行文件

make 會先將彙編和 C 源文件編譯為目標文件（.o），  
然後將它們link為最終的可執行文件（halt.coff）
最後，會將 COFF 格式轉換為 NOFF 格式，讓程序可以在 Nachos 模擬環境中執行。

其他程式運作流程與halt相似,簡單來說:先編譯為.o檔，由linker連結為.coff檔，再轉換成noff格式檔

In [ ]:
clean:
	$(RM) -f *.o *.ii
	$(RM) -f *.coff

distclean: clean
	$(RM) -f $(PROGRAMS)

# 若host can not be determined, 則透過@echo印出終止信息
unknownhost:
	@echo Host type could not be determined.
	@echo make is terminating.
	@echo If you are on an MFCF machine, contact the instructor to report this problem
	@echo Otherwise, edit Makefile.dep and try again.


make clean: 刪除由編譯⽣成的臨時檔

make distclean: 刪除Nachos可執⾏檔

##2. Implement four I/O system calls in NachOS

首先，進到 userprog 中的 syscall.h 把註解拿掉，程式碼如下

In [ ]:
#define SC_Halt 0
#define SC_Exit 1
#define SC_Exec 2
#define SC_Join 3
#define SC_Create 4
#define SC_Remove 5
#define SC_Open	6
#define SC_Read	7
#define SC_Write	8
#define SC_Seek 9
#define SC_Close	10
#define SC_ThreadFork 11
#define SC_ThreadYield 12
#define SC_ExecV 13
#define SC_ThreadExit 14
#define SC_ThreadJoin 15
#define SC_PrintInt 16
#define SC_Add 42
#define SC_MSG 100

然後進到 test 中的 start.S

In [ ]:
	.globl Open
	.ent	Open
Open:
	addiu $2,$0,SC_Open
	syscall
	j	$31
	.end Open

	.globl Write
	.ent	Write
Write:
	addiu $2,$0,SC_Write
	syscall
	j	$31
	.end Write

	.globl Read
	.ent	Read
Read:
	addiu $2,$0,SC_Read
	syscall
	j	$31
	.end Read

	.globl Close
	.ent	Close
Close:
	addiu $2,$0,SC_Close
	syscall
	j	$31
	.end Close

首先，宣告 .globl 函式名稱

這樣這個函式就能在廣域被調用(IBM的聲明文件寫的)

然後，宣告 .ent 函式名稱

這個應該是用來除錯用的

然後設定暫存器2儲存呼叫函式的位置(比如說呼叫 SC_Open 那就儲存 SC_Open 的位置到暫存器2中)，接著使用 systemcall 這樣 systemcall 執行時就能從暫存器2中找到要執行什麼

當執行完成後，就跳到暫存器31的位置，根據 mips register table 暫存器31應該是 return address 那就是跳到當初呼叫函式的下一行指令繼續完成其他操作

最後是 .end 函式名稱

用來宣告函式的末端

接著，進到 userprog 中的 exception.cc 將 SC_Open, SC_Write, SC_Read, SC_Close 做出來，具體而言就是讓各自功能呼叫各自對應的函式而已

In [ ]:
case SC_Open:
	//首先從暫存器4中讀取路徑在主記憶體中的位置
	val = kernel->machine->ReadRegister(4);
	{
		//進入主記憶體取出路徑
		char *filename = &(kernel->machine->mainMemory[val]);
		//主要是除錯功能
		DEBUG(dbgSys, "filename: " << filename << "\n");
		//然後呼叫SysOpen()來完成實際的處理
		status = SysOpen(filename);
		//主要是除錯功能
		DEBUG(dbgSys, "status: " << status << "\n");
		//將返回的值(打開檔案的編號)存到暫存器2中
		kernel->machine->WriteRegister(2, (int) status);
	}
	//下方應該是除錯用的
	kernel->machine->WriteRegister(PrevPCReg, kernel->machine->ReadRegister(PCReg));
	kernel->machine->WriteRegister(PCReg, kernel->machine->ReadRegister(PCReg) + 4);
	kernel->machine->WriteRegister(NextPCReg, kernel->machine->ReadRegister(PCReg)+4);
	return;
	ASSERTNOTREACHED();
	break;
case SC_Write:
	//首先從暫存器4中讀取路徑在主記憶體中的位置
	val = kernel->machine->ReadRegister(4);
	{
		//進入主記憶體取出路徑
		char *buffer = &(kernel->machine->mainMemory[val]);
		//然後呼叫SysWrite()來完成實際的處理
		int result = SysWrite(buffer,kernel->machine->ReadRegister(5),kernel->machine->ReadRegister(6));
		//將返回的值存到暫存器2中
		kernel->machine->WriteRegister(2, (int)result);
	}
	kernel->machine->WriteRegister(PrevPCReg, kernel->machine->ReadRegister(PCReg));
	kernel->machine->WriteRegister(PCReg, kernel->machine->ReadRegister(PCReg) + 4);
	kernel->machine->WriteRegister(NextPCReg, kernel->machine->ReadRegister(PCReg)+4);
	return;
	ASSERTNOTREACHED();
	break;
case SC_Read:
	val = kernel->machine->ReadRegister(4);
	{
		char *buffer = &(kernel->machine->mainMemory[val]);
		int result = SysRead(buffer,kernel->machine->ReadRegister(5),kernel->machine->ReadRegister(6));
		kernel->machine->WriteRegister(2, (int)result);
	}
	kernel->machine->WriteRegister(PrevPCReg, kernel->machine->ReadRegister(PCReg));
	kernel->machine->WriteRegister(PCReg, kernel->machine->ReadRegister(PCReg) + 4);
	kernel->machine->WriteRegister(NextPCReg, kernel->machine->ReadRegister(PCReg)+4);
	return;
	ASSERTNOTREACHED();
	break;
case SC_Close:
	val = kernel->machine->ReadRegister(4);
	{
		int result = SysClose((int) val);
		kernel->machine->WriteRegister(2, (int)result);
	}
	kernel->machine->WriteRegister(PrevPCReg, kernel->machine->ReadRegister(PCReg));
	kernel->machine->WriteRegister(PCReg, kernel->machine->ReadRegister(PCReg) + 4);
	kernel->machine->WriteRegister(NextPCReg, kernel->machine->ReadRegister(PCReg)+4);
	return;
	ASSERTNOTREACHED();
	break;

基本上就是參考 SC_CREATE 模仿出來的

接著進到 userprog 中的 ksyscall.h 去完成上面呼叫的程式，基本上就是呼叫位於 kernel 中 filesystem 之下的具體函式

In [ ]:
int SysOpen(char *filename) {
  return kernel->fileSystem->OpenAFile(filename);
}

int SysWrite(char *buffer, int size, int id) {
  return kernel->fileSystem->Write(buffer, size, id);
}

int SysRead(char *buffer, int size, int id) {
  return kernel->fileSystem->Read(buffer, size, id);
}

int SysClose(int id) {
  return kernel->fileSystem->Close(id);
}

最後，進到filesys 中的 filesys.h

In [ ]:
OpenFile* Open(char *name) {
    int fileDescriptor = OpenForReadWrite(name, FALSE);
    if (fileDescriptor == -1) return NULL;
    return new OpenFile(fileDescriptor);
}

OpenFileId OpenAFile(char *name) {
    //首先模仿 Open 函式打開檔案
    int fileDescriptor = OpenForReadWrite(name, FALSE);
    //如果打開失敗則回傳 -1
    if (fileDescriptor == -1) return -1;
    OpenFile *newFile = new OpenFile(fileDescriptor);
    int rfd = -1;
    //遍歷 OpenFileTable 尋找空閒的位置
    for (int i=0; i<20; i++) {
        //如果已經存在則回傳 -1 代表開啟失敗
        if (OpenFileTable[i] == newFile) {
            if (rfd != -1) OpenFileTable[rfd] = NULL;
            return -1;
        }
        //找到空間則將檔案放入 OpenFileTable
        if (OpenFileTable[i] == NULL && rfd == -1) {
            rfd = i;
        }
    }
    if (rfd == -1) return -1;
    OpenFileTable[rfd] = newFile;
    return rfd+1;
}


int Write(char *buffer, int size, OpenFileId id){
    //如果 id 有誤或者檔案大小有誤則回傳 -1 代表執行失敗
    if(id < 1||id > 20||size < 0) return -1;
    //將指定編號的檔案從 OpenFileTable 取出
    OpenFile *opFile = OpenFileTable[id-1];
    //如果找不到指定編號的檔案則回傳 -1 代表執行失敗
    if (opFile == NULL) return -1;
    //調用函式將檔案內容寫入主記憶體
    int numWritten = opFile->Write(buffer, size);
    //返回寫入字數
    return numWritten;
}
int Read(char *buffer, int size, OpenFileId id){
    //如果 id 有誤或者檔案大小有誤則回傳 -1 代表執行失敗
    if(id < 1||id > 20||size < 0) return -1;
    //將指定編號的檔案從 OpenFileTable 取出
    OpenFile *opFile = OpenFileTable[id-1];
    //如果找不到指定編號的檔案則回傳 -1 代表執行失敗
    if (opFile == NULL) return -1;
    //調用函式將檔案內容讀出主記憶體
    int numRead = opFile->Read(buffer, size);
    //返回讀取字數
    return numRead;
}
int Close(OpenFileId id){
    //如果 id 有誤則回傳 -1 代表執行失敗
    if(id < 1||id > 20) return -1;
    //將指定編號的檔案從 OpenFileTable 取出
    OpenFile *opFile = OpenFileTable[id-1];
    //如果找不到指定編號的檔案則回傳 -1 代表執行失敗
    if (opFile == NULL) return -1;
    //將指定編號的檔案從 OpenFileTable 清出
    OpenFileTable[id-1] = NULL;
    //返回 1 代表執行成功
    return 1;
}

##心得

林群佑:  

在這次作業中我遇上蠻多的困難像是在本地運行 NachOS 系統，雖然最後還是放棄本地運行，使用伺服器環境較為完整不須太多額外調整，然而在最後寫出程式時，還需要花很多時間去 lib 尋找具體要呼叫哪個函式來完成最終的動作也是不太友善，總之最後的引導希望可以多一點，

刁一平:

首次trace如此大量的code讓我感到overwhleming, 很多時候都要一行一行慢慢看
然後跟著function一直跑來跑去, 也了解到一些同步class的機制像是lock 跟semaphor中的P()與V(),雖然過程辛苦，但能夠理解system call的運作，不過我開始擔心後面的作業是否能夠負荷